In [1]:
import string
import pandas as pd
import nltk
import gensim
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from tqdm.notebook import tqdm

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim import corpora
from gensim.models import LdaModel

ModuleNotFoundError: No module named 'pyLDAvis'

In [ ]:
df_january = pd.read_csv('data/reddit_data/Submissions_2021-01_FilteredBySubreddit_GME.csv')
df_february = pd.read_csv('data/reddit_data/Submissions_2021-02_FilteredBySubreddit_GME.csv')
df_march = pd.read_csv('data/reddit_data/Submissions_2021-03_FilteredBySubreddit_GME.csv')

df_full = pd.concat([df_january, df_february, df_march], ignore_index=True)

start = '2021-01-01'  
end = '2021-01-15' 
df = df_full[(df_full['created_at'] >= start) & (df_full['created_at'] <= end)]

# df = pd.concat([df_january, df_february, df_march], ignore_index=True)

In [ ]:
custom_stopwords = ['nan', 'deleted', 'removed']

# Create a copy of the DataFrame to avoid warnings and preserve the original DataFrame
df_copy = df.copy()

# Combining 'title' and 'text'
df_copy['combined_text'] = df_copy['title'] + ' ' + df_copy['text']

# Lowercasing
df_copy['combined_text'] = df_copy['combined_text'].str.lower()

# Stripping punctuation and digits
df_copy['combined_text'] = df_copy['combined_text'].str.translate(str.maketrans('', '', string.punctuation + string.digits))

df_copy['combined_text'] = df_copy['combined_text'].astype(str)

df_copy['tokens'] = df_copy['combined_text'].apply(nltk.word_tokenize)

stop_words = set(stopwords.words('english'))
stop_words.update(custom_stopwords)

df_copy['tokens'] = df_copy['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

lemmatizer = WordNetLemmatizer()
df_copy['lemmatized'] = df_copy['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

df_copy['preprocessed_text'] = df_copy['lemmatized'].apply(lambda x: ' '.join(x))

In [ ]:
# Tokenize the text data (assuming it's already preprocessed and tokenized)
tokenized_text = df_copy['combined_text'].apply(lambda x: x.split())

# Create a dictionary mapping tokens to unique IDs
dictionary = corpora.Dictionary(tokenized_text)

# Create a corpus: a list of bags of words (token ID, token count)
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

In [ ]:
num_topics = 5
passes = 10
alpha = 'auto'
beta = 'auto'

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=passes,
    alpha=alpha,
    eta=beta,
    iterations=50 
)

for pass_num in range(passes):
    print(f"Pass {pass_num + 1}/{passes}")
    lda_model.update(corpus)
    print("Iteration complete")


Pass 1/10
Iteration complete
Pass 2/10
Iteration complete
Pass 3/10
Iteration complete
Pass 4/10
Iteration complete
Pass 5/10
Iteration complete
Pass 6/10
Iteration complete
Pass 7/10
Iteration complete
Pass 8/10
Iteration complete
Pass 9/10
Iteration complete
Pass 10/10
Iteration complete


In [ ]:
document_topics = [lda_model.get_document_topics(doc) for doc in corpus]

VISUALISATION

In [ ]:
vis_data = gensimvis.prepare(
    lda_model,  
    corpus,     
    dictionary, 
    sort_topics=False, # Do not sort topics
    mds='mmds',  # Multidimensional scaling method (customize as needed)
    R=30  # Number of terms to show for each topic 
)

# Customize the visualization appearance
pyLDAvis.display(vis_data, template_type='notebook')

/Users/User/Desktop/ds105/ds105a-project-data-dabblers/wordcloud/lib/python3.11/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(
